In [2]:

from glob import glob
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import scipy.signal as signal
from scipy.interpolate import interp1d
import os

In [117]:
reforecast = xr.open_dataset('data/S2S/reforecasts/2020-01-06/merged.nc')
reforecast = reforecast.sortby('lat').sortby('lon').sel({'lon':slice(220,290),'lat':slice(-40,40)})
reforecast = reforecast.assign_coords({'i':('lat',range(len(reforecast.lat))),'j':('lon',range(len(reforecast.lon)))})

In [60]:
lon,lat = np.meshgrid(reforecast.lon,reforecast.lat)

In [220]:
tropicalmask = reforecast.dslm[0,0,:,:].where(reforecast.lat>=-2).where(reforecast.lat<=2)
tropicalmask = xr.where(~np.isnan(tropicalmask),1,0)

watermask = reforecast.dslm[0,0,:,:].fillna(-9999)
watermask = xr.where(watermask==-9999,0,1)
watermask = watermask.differentiate('lon')!=0
watermask = watermask.where(watermask).ffill('lon').bfill('lon',limit=1)
x = xr.concat([watermask[i,:].dropna('lon').isel(lon=0).j for i in range(len(watermask.lat))],'lat')
watermask = xr.where(watermask==9999,1,0)
watermask[:,x[:]] = 1
watermask[23,[65,67]]=1
watermask[33,58]=1
watermask[37,58]=1
watermask[47,[58,59,60]]=1
watermask[48,56]=1
watermask[53,[48,49,50]]=1
watermask[55,[42,43,44,45]]=1
watermask[55,[42,43,44,45]]=1
watermask[56,[39,40]]=1
watermask[58,[35,36,37]]=1
watermask[62,[29,30,31,32]]=1
watermask[64,27]=1
watermask[74,19]=1
watermask = watermask.where(abs(reforecast.lat)>2).fillna(0)


coastmask_south = watermask.sel(lat=slice(-90,-2))
coastmask_north = watermask.sel(lat=slice(2,90))


In [226]:
tropicalmask.to_netcdf('data/TROPICALMASK.nc')
coastmask_north.to_netcdf('data/COASTNORTHMASK.nc')
coastmask_south.to_netcdf('data/COASTSOUTHMASK.nc')